<a href="https://colab.research.google.com/github/boxed-mikann/google_colab_ipynb/blob/main/BPEexercisesA2025_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **生物工学演習A後半　第４回**

このページは生物工学演習A後半の授業資料です。閲覧・共有は受講者限りとします。

<br>

なお、「オプション」となっている項目は応用問題です。難しければ必ずしも解答する必要はありませんが、解答は加点対象になります。

<br><br>


## **課題　オープンデータを用いた脳計量**


学術研究に用いたデータは世界中の誰でもが使えるように公開することが一般的になりつつあります。このようにして公開されたデータはオープンデータと呼ばれ、様々なオープンデータをまとめたサイトが存在します。

<br>

脳・神経科学関係のデータを公開する大手サイトの一つにOpenNeuro.orgがあります。OpenNeuroでは1,400を超えるデータセット、60,000人を超える被験者のデータが公開されています（2025年7月現在）。

https://openneuro.org

<br>

今回はここに公開されているヒト超高磁場（9.4T）MRI構造画像データセットを扱います。


UltraCortex: Submillimeter Ultra-High Field 9.4T Brain MR Image Collection and Manual Cortical Segmentations

Mahler et al., (2024) *bioRxiv/OpenNeuro*

https://openneuro.org/datasets/ds005216/versions/1.0.1


<br>

上記データセットでは、0.6-0.8mm等の高いボクセル解像度で撮影されたヒト脳のT1強調画像（sub-XX/ses-1/anat/XX.nii）や、大脳皮質・白質ボクセルを抽出してその位置を示したセグメンテーションデータ（derivatives/manual_segmentation/XX.nii）、また被験者の属性情報（participants.tsv; 被験者ID、年齢、性別）などが公開されています。

<br>

1.  **上記オープンデータについて、被験者1名を下記の基準で選んだ上で、サイトのGUIを用いてT1強調画像の断面図可視化を行ってください（スクリーンショットを解答）。ただし、解答は以下の条件に沿う形で進めてください。**

  *   横断面・冠状面・矢状面それぞれに小脳が明確に含まれるようにスライスを選択してください。
  *   自分の学籍番号の下2桁と同じ、もしくはそれに最も近い被験者IDのデータについて解答してください。
  *   選んだ被験者について、被験者ID、年齢、および性別も合わせて解答してください。

2.  **手動セグメンテーション（manual segmentation）が行われている被験者のうちの任意の1名のデータについて、抽出された左右大脳皮質のボクセル位置をプロットしてください。プロットは複数スライスについて表示し、脳の形の概要が判るようにしてください。**

3.  **手動セグメンテーションが行われている被験者12名全員のデータについて、左右大脳皮質の体積（立方ミリメートル単位）を求め、12名の平均と標準偏差を解答してください。**

4.  **ヒト大脳皮質1立方ミリメートルには平均約4万個の神経細胞、および約7億個のシナプス結合(\*1)があるとされています。3.で求めた皮質体積と合わせ、ヒト大脳皮質全体にはどの程度のシナプス結合があるか概算してください。また、その数と近年の基盤モデル・大規模言語モデル（LLM）等の巨大AIモデルの結合パラメータ数（情報が公開されている中で最大級のものはLlama 4のパラメータ数2T、T=trillions）を比較し、どちらがどの程度（何倍程度）多いかを考察してください。**

5.  （オプション）**上記データのderivatives/brainmasksには小脳なども含めた脳全体（全脳）のボクセル位置を示すマスクデータが入っています。このデータと被験者情報のクロス集計を行い、性別・年齢に関する全脳体積の統計的傾向を求めてください。**


<br>

\*1: Lennie P. (2003). The cost of cortical computation. *Current Biology*, 13(6), 493–497. https://doi.org/10.1016/s0960-9822(03)00135-0

<br>

ヒント：　以下のサンプルコードや図を参照しても構いません。ただし、コードを使う場合は各行が何をしているかを理解してから利用してください。


<br>




OpenNeuroのGUIを用いたT1強調画像のスクリーンショット例：

<img src="https://nishimotolab.org/images/seeds/T1_9T_sub10.png" width=100%>



準備
```python
##　注意：以下はGoogle Colabセッション中に1回行えば環境構築とダウンロードが実行されます。セッション中に2回以上実行する必要はありません。

# GitHubを使うために必要な環境を構築
!apt-get update && apt-get install -y git-annex
!git annex init

# GitHubを介して該当するOpenNeuroデータの情報（ディレクトリ構造・ファイルリスト）を取得
# （注意: ここではファイルのリストのみが取得され、「ファイル」にそれらが表示はされますがその実体はColabサーバにダウンロードされていません）
!git clone https://github.com/OpenNeuroDatasets/ds005216.git

# derivatives/manual_segmentation内のファイルを一括ダウンロード
# （注意: この作業を行うまではファイルはColabサーバにダウンロードされていません。）
%cd /content/ds005216
!git annex get derivatives/manual_segmentation

```

<br>

データ解析を行うサンプルコード

```python
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

fname = '/content/ds005216/derivatives/manual_segmentation/sub-20_ses-1_seg.nii'

# niiファイルをロード
niidata = nib.load(fname)

# ボクセルサイズ（mm）を取得
voxel_sizes = niidata.header.get_zooms()

# Segmentationされた3次元ボリュームを取得
data = niidata.get_fdata()

## ボリューム中の要素番号と脳部位の対応
## 2   Left-Cerebral-White-Matter    左大脳白質
## 3   Left-Cerebral-Cortex          左大脳皮質（灰白質）
## 41  Right-Cerebral-White-Matter   右大脳白質
## 42  Right-Cerebral-Cortex         右大脳皮質（灰白質）

# 一部のスライスを選択して表示用の2次元行列を作成
d = data[:, :, np.linspace(0,data.shape[2]-1,8, dtype=int)]
d = d.transpose(1,2,0)
d = np.flip(d,axis=0)
d2 = np.reshape(d,[d.shape[0],d.shape[1]*d.shape[2]])

# 左大脳白質（2）と左大脳皮質(3)を選択してマスクを作成
im = (d2 == 2) | (d2 == 3)

plt.figure(figsize=(15,15))
plt.imshow(im)
plt.show()

# 左大脳白質と左大脳皮質を選択し、ボクセル数と体積を求める
count = np.count_nonzero((data == 2) | (data == 3))
volume_in_mm3 = count*np.prod(voxel_sizes)
print("Voxel count:", count)
print("Voxel size (mm):", np.round(voxel_sizes,2))
print("Volume (mm^3):", round(volume_in_mm3))
```

<br>

左右大脳皮質の想定プロット例（抜粋）

<img src="https://nishimotolab.org/images/seeds/manual_segmentation.png" width=100%>

<br><br>

その他のヒント：

多数のファイルの一括処理を行う場合、「os」ライブラリなどの機能を使ってディレクトリ中のファイル一覧をリストとして扱うと効率的です。


*   python「os」ライブラリのドキュメントや利用サンプルを確認
*   AIに「ディレクトリ（フォルダ）中のファイル一覧をリストとして取得する」等のプロンプト指定を行って得られたサンプルコードを参照


<br>

